In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import sys

import floris.tools as wfct
from floris.utilities import Vec3  # to specify grid resolution

import pyamg

In [3]:
import os
if not os.path.isdir('figures'):
    os.makedirs('figures')

In [16]:
case = 'Layout_5x5_rev1.json'
R = 63.0
zhub = 125.99

xmax = 2520/R
ymin,ymax = -1260/R, 1260/R

# HARD-CODED PARAMS
debug = False
tol = 1e-6
savefloris = True

# u contour levels to plot
#ulevels = [0.7, 0.8, 0.9, 0.95, 0.98, 0.99]

## setup domain

In [12]:
xrange = (-10, xmax+10) # rotor radii
yrange = (ymin-1-buf, ymax+1+buf)
buf = 10  # lateral/vertical extent, in rotor radii
#spacing = 0.4 # grid spacing, in rotor radii
#spacing = 0.2 # grid spacing, in rotor radii
spacing = 0.1 # grid spacing, in rotor radii

groundeffect = True

In [13]:
if groundeffect:
    #zrange = (0, zhub+1+buf)
    zrange = (spacing, zhub+1+buf) # our domain does _not_ include boundary points
else:
    zrange = (zhub-1-buf, zhub+1+buf)
Nx = int((xrange[1]-xrange[0]) / spacing) + 1
Ny = int((yrange[1]-yrange[0]) / spacing) + 1
Nz = int((zrange[1]-zrange[0]) / spacing) + 1
N = Nx*Ny*Nz
print('Nx,Ny,Nz =',Nx,Ny,Nz)
x1 = (np.arange(Nx)*spacing + xrange[0]) * R
y1 = (np.arange(Ny)*spacing + yrange[0]) * R
z1 = (np.arange(Nz)*spacing + zrange[0]) * R

Nx,Ny,Nz = 601 621 1369


## calculate floris wake

In [21]:
fi = wfct.floris_interface.FlorisInterface(case)

In [ ]:
%%time
print('Calculating floris wake')

fi.floris.farm.flow_field.reinitialize_flow_field(
    with_resolution=Vec3(Nx,Ny,Nz),
    bounds_to_set=list(xrange)+list(yrange)+list(zrange),
)
fi.calculate_wake()

Calculating floris wake


In [ ]:
hor_plane = fi.get_hor_plane()
hor_plane.df[['u','v','w']] /= wspd

fig, ax = plt.subplots()
cm = wfct.visualization.visualize_cut_plane(hor_plane, ax=ax, cmap='Blues', minSpeed=vmin)
cb = fig.colorbar(cm)